In [1]:
import pandas as pd
import numpy as np
import sys
print('Python version:', sys.version)
print('Pandas version:', pd.__version__)
print('Numpy version:', np.__version__)

from pyecharts.charts import Scatter, Timeline
from pyecharts.globals import JsCode
from pyecharts import options as opts
from pyecharts.globals import ThemeType

Python version: 3.10.9 (tags/v3.10.9:1dd9be6, Dec  6 2022, 20:01:21) [MSC v.1934 64 bit (AMD64)]
Pandas version: 2.2.2
Numpy version: 2.1.1


In [44]:
data = pd.read_excel('./中国分省GDP数据.xlsx')

In [4]:
from pyecharts.globals import CurrentConfig  # 加载全局配置
CurrentConfig.ONLINE_HOST = "http://127.0.0.1:8000/assets/"  # 设定静态资源地址
from pyecharts.datasets import register_url  # 部分地图文件注册
# 注册本地服务器的地图数据源
register_url("http://127.0.0.1:8000/") 

In [17]:
import json
import os
filenames = os.listdir('./maps/')
shss = {'type': 'FeatureCollection','features':[]}
for i in filenames:
    with open(f"./maps/{i}",'r',encoding='utf-8') as f:
        data = json.loads(f.read())
        shss['features'].append(data)

In [18]:
target_data = """{
        "type": "FeatureCollection",
        "features": []}"""
target_data = json.loads(target_data)
list_region = os.listdir('./maps/')
name_list = []
for lr in list_region:
    with open(f"./maps/{lr}", 'r', encoding='utf-8') as f:
        temp = json.load(f)
        temp = temp['features']
        for tp in temp:
            name = tp['properties']['name']
            # 将area加入tp节点并传入target_data
            tp['properties']['area'] = '山河四省'
            target_data['features'].append(tp)
            name_list.append((name, np.random.randint(100, 5000)))

In [20]:
from pyecharts.charts import Map
maps = Map(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
maps.add_js_funcs(f"echarts.registerMap('山河四省', {target_data});")
# 引用添加的地图
maps.add("山河四省",[('河南省',100)],maptype='山河四省')
maps.render()

'C:\\Users\\wang\\Desktop\\pyLearn\\作业\\山河四省\\render.html'

In [21]:
dicts = {
    '华东':['上海市','江苏省','浙江省','安徽省','福建省','江西省','山东省','台湾省'],
    '华南':['广东省','广西壮族自治区','海南省','香港特别行政区','澳门特别行政区'],
    '华北':['北京市','天津市','河北省','山西省','内蒙古自治区'],
    '华中':['河南省','湖北省','湖南省'],
    '西南':['重庆市','四川省','贵州省','云南省','西藏自治区'],
    '西北':['陕西省','甘肃省','青海省','宁夏回族自治区','新疆维吾尔自治区'],
    '东北':['辽宁省','吉林省','黑龙江省']
}
dicts2 = {}
for k,v in dicts.items():
    dicts2.update({i:k for i in v})

In [56]:
data['province'] = data['province'].map(dicts2)

In [47]:
itemstyle = {
    'normal': {
        'borderWidth':1,
        'borderColor':JsCode("""new echarts.graphic.LinearGradient(0, 0, 1, 1, [
          { offset: 0, color: 'rgb(255, 191, 0)' },
          { offset: 1, color: 'rgb(224, 62, 76)' }
        ],)"""),
        'shadowColor':'green',
        'shadowBlur':1,
    }
}

In [61]:
from pyecharts.charts import Map,Timeline
timeline = Timeline()
for year in range(2014,2024):
    data_year = data.loc[data['date'] == year].groupby(by = 'province')['GDP'].sum().reset_index().values.tolist()
    maps = (Map(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
           .add(series_name = f'七大区{year}年GDP地图的绘制',data_pair=data_year,maptype='中国七大区',
                is_map_symbol_show=False,is_roam=False,
                itemstyle_opts=itemstyle,
                zoom=1.2
                )
            .set_global_opts(visualmap_opts=opts.VisualMapOpts(is_piecewise=True,
                             pieces=[
                                 {'min':300000,'color':'red'},
                              {'min':200000,'max':299999,'color':'yellow'},
                              {'min':100000,'max':199999,'color':'green'},
                              {'min':50000,'max':99999,'color':'pink'},
                              {'min':0,'max':49999,'color':'orange'}
                          ]),
                             title_opts=opts.TitleOpts(title = f'七大区{year}年GDP地图的绘制')
                             )
           )
    timeline.add(maps,year)
timeline.add_schema()
timeline.render()

'C:\\Users\\wang\\Desktop\\pyLearn\\作业\\山河四省\\render.html'

In [60]:
data[data['date'] == 2023].groupby(by = 'province')['GDP'].sum().reset_index().values.tolist()

[['东北', 59624.5],
 ['华东', 483668.6],
 ['华中', 164948.9],
 ['华北', 154767.3],
 ['华南', 170426.80000000002],
 ['西北', 73889.9],
 ['西南', 143605.8]]